<a href="https://colab.research.google.com/github/Nagalakshmi07/Labs-fmml/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [16]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [17]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [18]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [19]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [20]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [21]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [22]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [23]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [24]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results
Averaging the validation accuracy across multiple splits can indeed provide more consistent results. Here's why:


1. *Reducing variance*: By averaging the validation accuracy across multiple splits, you're reducing the variance of the estimate. This is because the validation accuracy for each split is an independent and identically distributed (i.i.d.) random variable. Averaging these variables reduces the variance of the estimate, making it more reliable.
2. *Improving robustness*: Averaging across multiple splits makes the estimate more robust to outliers or unusual splits. If one split has an unusually high or low validation accuracy, the average will be less affected by this outlier.
3. *Better representation of the data*: By using multiple splits, you're getting a better representation of the data. Each split provides a unique perspective on the data, and averaging across these perspectives gives a more comprehensive understanding of the model's performance.


However, there are some considerations to keep in mind:


1. *Computational cost*: Averaging across multiple splits requires more computational resources, as you need to train and evaluate the model on each split.
2. *Over-averaging*: If you average across too many splits, you might start to over-average, which can lead to a loss of important information. It's essential to find a balance between averaging and preserving the important characteristics of the data.


In general, averaging the validation accuracy across 3-5 splits is a good starting point. However, the optimal number of splits depends on the specific problem, dataset, and model architecture.


To further improve the consistency of your results, consider using techniques like:


1. *Stratified splitting*: Split the data in a way that preserves the class balance.
2. *K-fold cross-validation*: Use k-fold cross-validation to evaluate the model on multiple splits and average the results.
3. *Bootstrapping*: Use bootstrapping to resample the data and evaluate the model on multiple bootstrap samples.
2. Does it give more accurate estimate of test accuracy?
K-fold cross-validation can provide a more accurate estimate of test accuracy compared to other methods, such as:


1. *Holdout method*: Splitting the data into training and testing sets only once can lead to an inaccurate estimate of test accuracy, as the model's performance can vary depending on the specific split.
2. *Single train-test split*: Using a single train-test split can also lead to an inaccurate estimate of test accuracy, as the model's performance can be influenced by the specific split.


K-fold cross-validation can provide a more accurate estimate of test accuracy for several reasons:


1. *Reducing variance*: By averaging the test accuracy across multiple folds, k-fold cross-validation reduces the variance of the estimate, providing a more reliable estimate of test accuracy.
2. *Increasing robustness*: K-fold cross-validation increases the robustness of the estimate by evaluating the model on multiple subsets of the data, which can help to identify and mitigate overfitting.
3. *Better representation of the data*: By using multiple folds, k-fold cross-validation provides a better representation of the data, as each fold is used for both training and testing.


However, it's essential to note that k-fold cross-validation is not foolproof, and there are some scenarios where it may not provide an accurate estimate of test accuracy:


1. *Small datasets*: If the dataset is very small, k-fold cross-validation may not provide a reliable estimate of test accuracy.
2. *Highly imbalanced datasets*: If the dataset is highly imbalanced, k-fold cross-validation may not provide a reliable estimate of test accuracy, as the model's performance can be biased towards the majority class.
3. *Models with high variance*: If the model has high variance, k-fold cross-validation may not provide a reliable estimate of test accuracy, as the model's performance can vary significantly across different folds.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
The number of iterations, also known as the number of folds (K) in K-fold cross-validation, can significantly impact the estimate of the model's performance.


_Effect of increasing the number of iterations:_

1. *Reduced variance*: As the number of iterations increases, the variance of the estimate decreases. This means that the estimate becomes more stable and reliable.
2. *Improved accuracy*: With more iterations, the estimate of the model's performance becomes more accurate. This is because the model is evaluated on more subsets of the data, providing a more comprehensive understanding of its performance.
3. *Increased computational cost*: As the number of iterations increases, the computational cost also increases. This is because the model needs to be trained and evaluated on more subsets of the data.


_Do we get a better estimate with higher iterations?_

Yes, in general, increasing the number of iterations can provide a better estimate of the model's performance. However, there are some limitations to consider:


1. *Diminishing returns*: As the number of iterations increases, the improvement in the estimate's accuracy may become smaller. This means that increasing the number of iterations beyond a certain point may not provide significant benefits.
2. *Computational cost*: As mentioned earlier, increasing the number of iterations can significantly increase the computational cost. This may be a limiting factor, especially when working with large datasets or complex models.
3. *Overfitting*: If the number of iterations is too high, the model may start to overfit to the training data. This means that the model becomes too specialized to the training data and fails to generalize well to new, unseen data.


In practice, the optimal number of iterations depends on the specific problem, dataset, and model. A common approach is to start with a relatively small number of iterations (e.g., 5-10) and increase it until the estimate's accuracy plateaus or the computational cost becomes prohibitively expensive.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
When dealing with a very small train dataset or validation dataset, increasing the iterations can help to some extent, but it's not a silver bullet.


Here are some pros and cons to consider:


Pros:


1. _Reduced variance_: Increasing the iterations can help reduce the variance of the estimate, making it more stable and reliable.
2. _Improved accuracy_: With more iterations, the model can learn from the same data multiple times, which can lead to improved accuracy.


Cons:


1. _Overfitting_: Increasing the iterations can lead to overfitting, especially when the dataset is very small. The model may become too specialized to the training data and fail to generalize well to new, unseen data.
2. _Limited generalizability_: Even with more iterations, a small dataset may not provide enough information for the model to learn generalizable patterns.
3. _Computational cost_: Increasing the iterations can significantly increase the computational cost, which may be a limiting factor.


To deal with a very small train dataset or validation dataset, consider the following strategies:


1. _Data augmentation_: If possible, generate more data through augmentation techniques, such as rotation, scaling, or flipping.
2. _Transfer learning_: Use pre-trained models and fine-tune them on your small dataset.
3. _Regularization techniques_: Use regularization techniques, such as L1 or L2 regularization, to prevent overfitting.
4. _Ensemble methods_: Use ensemble methods, such as bagging or boosting, to combine the predictions of multiple models.
5. _Collect more data_: If possible, collect more data to increase the size of the train and validation datasets.


In summary, while increasing the iterations can help with a small dataset, it's essential to consider other strategies to improve the model's performance and generalizability.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.